<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/Autoencoder_Recommender/Content_Based_Recommerders_TF_IDF_BERT_y_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Content Based Recommerders: TF-IDF, BERT y Autoencoders**

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Introducción y Configuración Inicial</a>  
2. <a href="#item32">Preparación del Entorno y Descarga del Dataset</a>  
3. <a href="#item34">Análisis Exploratorio de Datos</a>  
4. <a href="#item34">Recomendaciones Basadas en Contenido con TF-IDF</a>
5. <a href="#item34">Recomendaciones Basadas en Contenido con BERT</a>
6. <a href="#item34">Recomendaciones Basadas en Contenido con Autoencoders</a>
7. <a href="#item34">Conclusiones</a>



## 1. Introducción

En este laboratorio, exploraremos cómo generar recomendaciones de artículos basadas en contenido utilizando tres métodos distintos:

- TF-IDF (Term Frequency-Inverse Document Frequency)
- BERT (Bidirectional Encoder Representations from Transformers)
- Autoencoders (Redes Neuronales para Aprendizaje No Supervisado)

Cada método nos permitirá transformar textos en representaciones numéricas que capturan diferentes aspectos de su contenido, lo que nos ayudará a medir similitudes y generar recomendaciones personalizadas.



**Objetivos del laboratorio:**

Implementar sistemas de recomendación basados en contenido utilizando TF-IDF, BERT y Autoencoders.
Comprender las ventajas y limitaciones de cada enfoque.
Visualizar las relaciones entre los artículos mediante reducción de dimensionalidad.

## 2. Preparación del Entorno y Descarga del Dataset


Comenzamos configurando el entorno y descargando el dataset necesario para nuestro análisis.

**Instalación de Dependencias**

In [2]:
# Instalar la API de Kaggle
!pip install kaggle

# Subir el archivo kaggle.json con las credenciales de Kaggle
from google.colab import files
files.upload()  # Sube tu archivo kaggle.json aquí

# Configurar la API de Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


**Descarga y Preparación del Dataset**

Descargamos el dataset de artículos de The Guardian desde Kaggle y lo cargamos en un DataFrame de pandas.

In [ ]:
# Descargar el dataset
!kaggle datasets download -d adityakharosekar2/guardian-news-articles

# Descomprimir el archivo descargado
!unzip guardian-news-articles.zip

# Cargar el dataset en un DataFrame
import pandas as pd

data = pd.read_csv('guardian_articles.csv')

# Seleccionar las columnas relevantes y las primeras 1000 filas para agilizar el procesamiento
data = data[['webTitle', 'bodyContent']].head(1000)

# Mostrar las primeras filas del DataFrame
data.head()



## 3. Análisis Exploratorio de Datos

Antes de proceder, es importante entender el contenido y la estructura de nuestro dataset.

**Verificación de Datos Nulos**

In [ ]:
# Verificar valores nulos
print(data.isnull().sum())


Observamos que la columna bodyContent puede tener valores nulos, los cuales manejaremos a continuación.

**Limpieza de Datos**

In [ ]:
# Rellenar valores nulos en 'bodyContent' con cadenas vacías
data['bodyContent'] = data['bodyContent'].fillna('')


**Distribución de la Longitud de los Artículos**

Analizamos la longitud de los artículos para entender la variabilidad en el contenido.

In [ ]:
# Añadir una columna con la longitud de cada artículo
data['text_length'] = data['bodyContent'].apply(len)

# Visualizar la distribución de la longitud
import matplotlib.pyplot as plt

plt.hist(data['text_length'], bins=30, edgecolor='black')
plt.title('Distribución de la Longitud de los Artículos')
plt.xlabel('Longitud del Texto')
plt.ylabel('Frecuencia')
plt.show()


## 4. Recomendaciones Basadas en Contenido con TF-IDF

**¿Qué es TF-IDF?**

TF-IDF es una técnica que combina la frecuencia de un término en un documento (TF) con la frecuencia inversa del término en el corpus (IDF) para calcular la importancia relativa de palabras en un documento específico.

**Implementación de TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear una instancia del vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Ajustar y transformar los textos
tfidf_matrix = vectorizer.fit_transform(data['bodyContent'])

print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)


La matriz TF-IDF resultante tiene una fila por artículo y una columna por cada término del vocabulario.

**Cálculo de la Similitud del Coseno**

Utilizamos la similitud del coseno para medir qué tan similares son dos artículos en base a sus vectores TF-IDF.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la matriz de similitud
similarity_matrix_tfidf = cosine_similarity(tfidf_matrix)


**Función para Recomendar Artículos**

Definimos una función que, dado un índice de artículo, recomiende los artículos más similares.

In [ ]:
def recommend_articles_tfidf(article_index, similarity_matrix, data, top_n=5):
    # Obtener los índices de los artículos más similares
    similar_indices = similarity_matrix[article_index].argsort()[-top_n-1:-1][::-1]

    # Extraer los títulos de los artículos recomendados
    recommended_articles = data.iloc[similar_indices]

    print("Artículo seleccionado:")
    print(data.iloc[article_index]['webTitle'], "\n")

    print("Artículos recomendados basados en TF-IDF:")
    for idx, row in recommended_articles.iterrows():
        print("-", row['webTitle'])


**Ejemplo de Recomendación**

Probamos la función con un artículo específico.

In [16]:
# Elegir un artículo de ejemplo
article_index = 10

# Generar recomendaciones
recommend_articles_tfidf(article_index, similarity_matrix_tfidf, data)


## 5. Recomendaciones Basadas en Contenido con BERT

**¿Qué es BERT?**

BERT es un modelo de lenguaje desarrollado por Google que utiliza transformadores para comprender el contexto bidireccional de las palabras en un texto, generando embeddings contextuales de alta calidad.

**Implementación de Embeddings con BERT**

Instalación de Sentence Transformers

In [ ]:
!pip install sentence-transformers

**Generación de Embeddings**


In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para los textos
embeddings = model.encode(data['bodyContent'], show_progress_bar=True)

print("Dimensiones de los embeddings:", embeddings.shape)


Cada artículo ahora está representado por un vector de dimensiones fijas que captura su significado semántico.

**Cálculo de la Similitud del Coseno**

In [ ]:
# Calcular la matriz de similitud
similarity_matrix_bert = cosine_similarity(embeddings)


**Función para Recomendar Artículos**

Utilizamos una función similar a la anterior, adaptada para los embeddings de BERT.

In [38]:
def recommend_articles_bert(article_index, similarity_matrix, data, top_n=5):
    # Obtener los índices de los artículos más similares
    similar_indices = similarity_matrix[article_index].argsort()[-top_n-1:-1][::-1]

    # Extraer los títulos de los artículos recomendados
    recommended_articles = data.iloc[similar_indices]

    print("Artículo seleccionado:")
    print(data.iloc[article_index]['webTitle'], "\n")

    print("Artículos recomendados basados en BERT:")
    for idx, row in recommended_articles.iterrows():
        print("-", row['webTitle'])


**Ejemplo de Recomendación**

In [ ]:
# Elegir un artículo de ejemplo
article_index = 10

# Generar recomendaciones
recommend_articles_bert(article_index, similarity_matrix_bert, data)


## 6. Recomendaciones Basadas en Contenido con Autoencoders

**¿Qué es un Autoencoder?**
Un autoencoder es una red neuronal que aprende a copiar su entrada a su salida. Consta de dos partes:

**Encoder:** Comprime la entrada en una representación latente de menor dimensión.
**Decoder:** Reconstruye la entrada original a partir de la representación latente.
Al entrenar el autoencoder, la red aprende a captar las características más importantes de los datos.

**Preprocesamiento Avanzado de Textos**


**Eliminación de Stopwords y Tokenización en Trigramas**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenizar el texto
    tokens = word_tokenize(text.lower())
    # Eliminar stopwords y palabras no alfanuméricas
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Aplicar el preprocesamiento
data['processed_text'] = data['bodyContent'].apply(preprocess_text)


**Construcción del Vocabulario con Trigramas**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear una instancia del vectorizador de conteo
vectorizer = CountVectorizer(ngram_range=(3,3), max_features=5000)

# Ajustar y transformar los textos procesados
vocab_matrix = vectorizer.fit_transform(data['processed_text'])

print("Dimensiones de la matriz de vocabulario:", vocab_matrix.shape)


**Construcción y Entrenamiento del Autoencoder**


Definición del Modelo

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

input_dim = vocab_matrix.shape[1]
encoding_dim = 100  # Dimensión de la representación latente

# Capa de entrada
input_layer = Input(shape=(input_dim,))

# Codificación
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decodificación
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Modelo autoencoder
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Modelo encoder (para obtener las representaciones latentes)
encoder = Model(inputs=input_layer, outputs=encoded)


**Compilación y Entrenamiento**

In [ ]:
# Compilar el modelo
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Convertir la matriz de vocabulario a array denso
vocab_matrix_dense = vocab_matrix.toarray()

# Entrenar el modelo
autoencoder.fit(vocab_matrix_dense, vocab_matrix_dense,
                epochs=20,
                batch_size=32,
                shuffle=True)


**Obtención de Representaciones Latentes**

In [ ]:
# Obtener las representaciones latentes
latent_representations = encoder.predict(vocab_matrix_dense)

print("Dimensiones de las representaciones latentes:", latent_representations.shape)


**Cálculo de la Similitud del Coseno**

In [ ]:
# Calcular la matriz de similitud
similarity_matrix_autoencoder = cosine_similarity(latent_representations)


**Función para Recomendar Artículos**

In [ ]:
def recommend_articles_autoencoder(article_index, similarity_matrix, data, top_n=5):
    # Obtener los índices de los artículos más similares
    similar_indices = similarity_matrix[article_index].argsort()[-top_n-1:-1][::-1]

    # Extraer los títulos de los artículos recomendados
    recommended_articles = data.iloc[similar_indices]

    print("Artículo seleccionado:")
    print(data.iloc[article_index]['webTitle'], "\n")

    print("Artículos recomendados basados en Autoencoder:")
    for idx, row in recommended_articles.iterrows():
        print("-", row['webTitle'])


**Ejemplo de Recomendación**

In [ ]:
# Elegir un artículo de ejemplo
article_index = 10

# Generar recomendaciones
recommend_articles_autoencoder(article_index, similarity_matrix_autoencoder, data)


## 7. Visualización de Clusters con t-SNE

**¿Qué es t-SNE?**
t-SNE es una técnica para reducir la dimensionalidad de datos de alta dimensión, permitiendo visualizar patrones y clusters en un espacio de dos o tres dimensiones.

**Implementación de t-SNE**

Aplicando t-SNE a las Representaciones Latentes

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

# Reducir la dimensionalidad con t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(latent_representations)


**Visualización**

In [ ]:
# Crear un DataFrame con los resultados de t-SNE
tsne_df = pd.DataFrame(tsne_results, columns=['Dim1', 'Dim2'])
tsne_df['title'] = data['webTitle']

# Graficar los resultados
plt.figure(figsize=(12,8))
plt.scatter(tsne_df['Dim1'], tsne_df['Dim2'], alpha=0.7)
plt.title('Visualización de Artículos con t-SNE')
plt.xlabel('Dimensión 1')
plt.ylabel('Dimensión 2')
plt.show()


Podemos observar agrupaciones de artículos que son similares según las representaciones aprendidas por el autoencoder.

## 8. Conclusiones


En este laboratorio, hemos implementado tres enfoques distintos para generar recomendaciones basadas en contenido:

**TF-IDF:** Un método sencillo y eficiente que considera la frecuencia de términos. Sin embargo, no captura relaciones semánticas profundas.
**BERT:** Proporciona embeddings contextuales que reflejan mejor el significado del texto, mejorando la calidad de las recomendaciones.
**Autoencoders:** Aprenden representaciones latentes que pueden captar patrones complejos en los datos, ofreciendo una alternativa poderosa para sistemas de recomendación.



**Reflexiones finales:**

- Calidad de las Recomendaciones: BERT y los Autoencoders suelen proporcionar recomendaciones más precisas que TF-IDF debido a su capacidad para entender el contexto y las relaciones semánticas.

- Costo Computacional: TF-IDF es menos costoso computacionalmente en comparación con BERT y los Autoencoders.

- Aplicabilidad: La elección del método depende de los recursos disponibles y de la complejidad requerida en las recomendaciones.

---
## Gracias por completar este laboratorio!